In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    scipy-1.4.1                |   py36h921218d_0        18.9 MB  conda-forge
    lib

In [5]:
!conda install -c conda-forge lxml --yes
import lxml.html as lh
print('Library imported')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    lxml-4.4.1                 |   py36h7ec2d77_0         1.6 MB  conda-forge

The following packages will be UPDATED:

    lxml: 4.2.5-py37hefd8a0e_0 --> 4.4.1-py36h7ec2d77_0 conda-forge


lxml-4.4.1           | 1.6 MB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Library imported


## Scraping Scraping the Toronto Wikipedia Page

In [6]:
#Toronto Wikipedia link
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#getting content from the html page
page_content = lh.fromstring(requests.get(url).content)
#accessing the table data
table_data = page_content.xpath("//tr")

## Generating Toronto Neighbourhood data

In [7]:
toronto_data=[]
col_ind=0

#Creating header with empty list to add list of values under each column, replace new line character
for th in table_data[0]:
    col_ind+=1
    header=str(th.text_content()).replace('\n','')
    print (col_ind,header)
    toronto_data.append((header,[]))
    #print(toronto_data)

1 Postcode
2 Borough
3 Neighbourhood


In [8]:
# range starts from 1, since header is handled
for td in range(1,len(table_data)):
    row=table_data[td]
    
    #Exclude if the row length is not 3
    if len(row) != 3:
      break
    
    col_ind=0
  
    #Iterating through each element of the row, replace new line character
    for val in row.iterchildren():
        data=str(val.text_content()).replace('\n','')
        #Append the data after header
        toronto_data[col_ind][1].append(data)
        #Increment i for next column
        col_ind+=1

#print(toronto_data)
#print(len(toronto_data))
#print([len(rows) for (i,rows) in toronto_data])

## Creating Data frame from the List of headers and rows of Toronto Neighbourhood data


In [9]:
#Creating a Data Frame
toronto_struct={header:rows for (header,rows) in toronto_data}
toronto_df=pd.DataFrame(toronto_struct)

#Dropping rows with Borough value of Not assigned and resetting the index
toronto_df = toronto_df.drop(toronto_df.index[toronto_df['Borough'] == 'Not assigned']).reset_index(drop = True)

#print(toronto_df.shape)
toronto_df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


## Grouping the data

In [10]:
#Grouping Neighbourhoods by Postal codes
toronto_df = toronto_df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()

#Replacing Neighbourhood with Not Assigned in column Borough
toronto_df.loc[toronto_df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = toronto_df['Borough']

#print(toronto_df[toronto_df['Borough'] == 'Queen\'s Park'])
toronto_df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [11]:
toronto_df.shape

(103, 3)

In [12]:
#Creating a csv file Neighbourhood data to use in Part2 of the project
toronto_df.to_csv(r'toronto_data.csv', index=False)